### 库导入

In [35]:
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import threading
import time

In [36]:
from lib.imgproc import to_jpeg_bytes, process_frame

### 自定义函数

In [37]:
# 关闭按钮逻辑
def close_video(b):
    global playing
    playing = False
    toggle_button.value = False
    toggle_button.disabled = True
    close_button.disabled = True
    slider_red1_h.disabled = True
    slider_red2_h.disabled = True
    slider_red1_s.disabled = True
    slider_red2_s.disabled = True
    slider_red1_v.disabled = True
    slider_red2_v.disabled = True
    cap.release()
    # 清除画面
    img_processed.value = b''
    img_mask.value = b''

In [38]:
# 播放/暂停按钮
# 引入全局变量 -> 获取change最新的值并赋值 -> 更新图标 + 继续播放/暂停
def toggle_play(change):
    global playing
    playing = change['new']# 获取change最新的值
    toggle_button.icon = 'pause' if playing else 'play'
    if playing:
        threading.Thread(target=play_video, daemon=True).start()

In [39]:
# playing为1，连续循环播放图片
def play_video():
    global playing
    while playing:
        with frame_lock:
            ret, frame = cap.read()
            if not ret:
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)# 播放完毕，从头开始重新播放
                continue
            temp = [
                    [slider_red1_h.value[0], slider_red1_s.value[0], slider_red1_s.value[0]],
                    [slider_red1_h.value[1], slider_red1_s.value[1], slider_red1_s.value[1]],
                    [slider_red2_h.value[0], slider_red2_s.value[0], slider_red2_v.value[0]],
                    [slider_red2_h.value[1], slider_red2_s.value[1], slider_red2_s.value[1]],
            ]
            draw ,mask = process_frame(frame,temp,lowest=slider_lowest.value)
            img_processed.value = to_jpeg_bytes(draw, is_mask=False)
            img_mask.value = to_jpeg_bytes(mask, is_mask=True)
        #----------------控制播放速度------------------------
        time.sleep(1/frame_rate.value)

In [40]:
# 滑块变化时更新当前帧
def refresh_on_change(change=None):
    with frame_lock:
        # 回退一帧，因为你用play_video函数读取了一帧，cv2.CAP_PROP_POS_FRAMES++,所以这里需要回退一下
        current = cap.get(cv2.CAP_PROP_POS_FRAMES)
        cap.set(cv2.CAP_PROP_POS_FRAMES, max(0, current - 1))
        ret, frame = cap.read()
        if ret:
            temp = [
                    [slider_red1_h.value[0], slider_red1_s.value[0], slider_red1_s.value[0]],
                    [slider_red1_h.value[1], slider_red1_s.value[1], slider_red1_s.value[1]],
                    [slider_red2_h.value[0], slider_red2_s.value[0], slider_red2_v.value[0]],
                    [slider_red2_h.value[1], slider_red2_s.value[1], slider_red2_s.value[1]],
            ]
            draw ,mask = process_frame(frame,temp,lowest=slider_lowest.value)
            img_processed.value = to_jpeg_bytes(draw, is_mask=False)
            img_mask.value = to_jpeg_bytes(mask, is_mask=True)

### 全局变量定义

In [41]:
playing = False
frame_lock = threading.Lock()

### 读取视频

In [42]:
cap = cv2.VideoCapture("video/red-green.mp4")
if not cap.isOpened():
    raise IOError("无法打开视频文件")

### 控件定义

In [43]:
# -------------------------------图像输出--------------------------
# 掩膜图像 + 处理后的图像
# 格式为 jpg , 大小为640*480
img_processed = widgets.Image(format='jpg', width=640, height=480)
img_mask = widgets.Image(format='jpg', width=640, height=480)

In [44]:
# -------------------------------控制按钮-------------------------
# 切换按钮 + 停止按钮
toggle_button = widgets.ToggleButton(description="Play/Pause", value=False, icon='play')
close_button = widgets.Button(description="关闭", icon='times', button_style='danger')
toggle_button.observe(toggle_play, names='value')
close_button.on_click(close_video)

In [52]:
# -------------------------------滑块控件------------------------
slider_lowest = widgets.IntSlider(value=30, min=0, max=100, description='lowest:')
frame_rate = widgets.IntSlider(value=30, min=0, max=200, description='frame rate:')
slider_red2_v = widgets.IntRangeSlider(value=[80, 255], min=0, max=255, description='red2 V:')
slider_red1_h = widgets.IntRangeSlider(value=[0, 15], min=0, max=180, description='red1 H:')
slider_red1_s = widgets.IntRangeSlider(value=[80, 255], min=0, max=255, description='red1 S:')
slider_red1_v = widgets.IntRangeSlider(value=[80, 255], min=0, max=255, description='red1 V:')
slider_red2_h = widgets.IntRangeSlider(value=[165, 180], min=0, max=180, description='red2 H:')
slider_red2_s = widgets.IntRangeSlider(value=[80, 255], min=0, max=255, description='red2 S:')
slider_red2_v = widgets.IntRangeSlider(value=[80, 255], min=0, max=255, description='red2 V:')

### 布局

In [53]:
group0 = widgets.HBox([toggle_button, close_button])
group1 = widgets.HBox([slider_red1_h, slider_red2_h])
group2 = widgets.HBox([slider_red1_s, slider_red2_s])
group3 = widgets.HBox([slider_red1_v, slider_red2_v])
controls = widgets.VBox([group0, frame_rate,group1, group2, group3])
image = widgets.HBox([img_processed, img_mask])

### 绑定回调函数

In [54]:
for s in [slider_lowest, slider_red1_h, slider_red2_h, slider_red1_s ,slider_red2_s , slider_red1_v,slider_red2_v]:
    # 当s.value变化时，调用refresh_on_change函数
    s.observe(refresh_on_change, names='value')

In [55]:
display(controls, image)
refresh_on_change()

In [20]:
cap.release()